In [1]:
using DataFrames;

In [2]:
using LinearAlgebra;

In [3]:
TOL = 1e-4

0.0001

In [4]:
tensor_df = readtable("tensor_head.csv");
for i in 1:size(tensor_df,1)
    curr_row = tensor_df[i,1:9];
    T_flat = [x for x in curr_row]
    T = transpose(reshape(T_flat, 3, 3))
    println(T)
    println("*****")
end

┌ Warning: readtable is deprecated, use CSV.read from the CSV package instead
│   caller = top-level scope at In[4]:1
└ @ Core In[4]:1


[-11.317 -10.12 2.8978; 45.466 86.037 -433.36; 0.028671 38.248 -62.274]
*****
[163.09 29.24 -198.21; 122.05 21.069 -89.915; 457.34 76.642 -172.86]
*****
[-44.434 189.45 81.13; 78.246 -45.926 -164.95; 32.59 -302.35 34.861]
*****
[390.12 53.034 296.01; 102.86 -15.358 87.843; -477.58 -46.895 -226.83]
*****
[163.72 117.25 -213.88; 154.44 133.69 -146.44; 259.4 320.89 -207.33]
*****
[45.593 21.837 -168.21; 166.08 137.2 -311.68; 77.686 141.76 -138.29]
*****
[-5.4906 -4.301 -34.579; 86.754 114.93 -469.75; 4.5217 59.763 -111.79]
*****
[5.054 -2.9375 -93.008; 132.55 108.53 -363.45; 20.453 91.849 -116.17]
*****
[74.973 65.682 -172.28; 157.06 152.09 -268.06; 119.14 209.27 -175.85]
*****


In [7]:
A = [-11.317 -10.12 2.8978; 45.466 86.037 -433.36; 0.028671 38.248 -62.274]

3×3 Array{Float64,2}:
 -11.317     -10.12      2.8978
  45.466      86.037  -433.36  
   0.028671   38.248   -62.274 

In [8]:
foo = eigen((A + A')/2)

Eigen{Float64,Float64,Array{Float64,2},Array{Float64,1}}
eigenvalues:
3-element Array{Float64,1}:
 -199.80875227189026 
  -11.440926915527712
  223.69567918741873 
eigenvectors:
3×3 Array{Float64,2}:
 -0.0598078   0.996509   -0.0582456
  0.570036   -0.0138046  -0.821504 
  0.81944     0.0823344   0.56722  

In [9]:
foo.values

3-element Array{Float64,1}:
 -199.80875227189026 
  -11.440926915527712
  223.69567918741873 

In [10]:
clamp.(foo.values,0, 10)

3-element Array{Float64,1}:
  0.0
  0.0
 10.0

In [30]:
#mutates out_Q and out_D_clamped
function get_tensor_warp!(T, e_min, e_max, out_Q, out_D_clamped)
    S = (T + T')/2; #S is the symmetric part of T
    eigenDecomp = eigen(S);
    out_Q[:] = eigenDecomp.vectors
    for lambda in eigenDecomp.values
        #TODO: reject values too close to zero?
        if lambda < 0 #reject negative
            return false;
        end
    end
    #clamped_eigvals = clamp.(eigenDecomp.values,0,10)
    clamped_eigvals = eigenDecomp.values #no clamping
    out_D_clamped[:] = Diagonal(clamped_eigvals)
    return true;
end

get_tensor_warp! (generic function with 1 method)

In [12]:
Q = Array{Float64,2}(undef, 3,3)

3×3 Array{Float64,2}:
 6.93479e-310  6.93479e-310  6.93479e-310
 6.93479e-310  6.93479e-310  0.0         
 6.93479e-310  6.93479e-310  0.0         

In [13]:
D_clamped = Array{Float64,2}(undef, 3,3)

3×3 Array{Float64,2}:
 6.93479e-310  4.94066e-324  6.93479e-310
 6.93479e-310  6.93479e-310  1.97626e-323
 6.93479e-310  6.93479e-310  1.97626e-323

In [14]:
fill!(Q,NaN)

3×3 Array{Float64,2}:
 NaN  NaN  NaN
 NaN  NaN  NaN
 NaN  NaN  NaN

In [15]:
fill!(D_clamped,NaN)

3×3 Array{Float64,2}:
 NaN  NaN  NaN
 NaN  NaN  NaN
 NaN  NaN  NaN

In [34]:
A

3×3 Array{Float64,2}:
 -11.317     -10.12      2.8978
  45.466      86.037  -433.36  
   0.028671   38.248   -62.274 

In [35]:
eigs_A_symm = eigen((A + A')/2)

Eigen{Float64,Float64,Array{Float64,2},Array{Float64,1}}
eigenvalues:
3-element Array{Float64,1}:
 -199.80875227189026 
  -11.440926915527712
  223.69567918741873 
eigenvectors:
3×3 Array{Float64,2}:
 -0.0598078   0.996509   -0.0582456
  0.570036   -0.0138046  -0.821504 
  0.81944     0.0823344   0.56722  

In [16]:
get_tensor_warp!(A, -10, 10, Q, D_clamped)

false

In [21]:
B = [1 -1 0; -1 2 -1; 0 -1 1]

3×3 Array{Int64,2}:
  1  -1   0
 -1   2  -1
  0  -1   1

In [22]:
eigs_B_symm = eigen((B + B')/2)

Eigen{Float64,Float64,Array{Float64,2},Array{Float64,1}}
eigenvalues:
3-element Array{Float64,1}:
 2.6645352591003757e-15
 1.0000000000000018    
 3.0                   
eigenvectors:
3×3 Array{Float64,2}:
 0.57735  -0.707107     -0.408248
 0.57735   9.42055e-16   0.816497
 0.57735   0.707107     -0.408248

In [31]:
get_tensor_warp!(B, -10, 10, Q, D_clamped)

true

In [32]:
Q

3×3 Array{Float64,2}:
 0.57735  -0.707107     -0.408248
 0.57735   9.42055e-16   0.816497
 0.57735   0.707107     -0.408248

In [33]:
D_clamped

3×3 Array{Float64,2}:
 2.66454e-15  0.0  0.0
 0.0          1.0  0.0
 0.0          0.0  3.0